# Clustering Crypto

In [38]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [39]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('crypto_data.csv')
crypto_df.head(5)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [40]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading'] == True]

In [41]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns ='IsTrading')

In [42]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
crypto_df.isnull().sum()

Unnamed: 0         0
CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [43]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"]> 0]
crypto_df.head(5)

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [44]:
# Create a new DataFrame that holds only the cryptocurrencies names.
CoinName_df= pd.DataFrame(crypto_df['CoinName']).reset_index(drop=True)
CoinName_df

,CoinName
0,42 Coin
1,404Coin
2,EliteCoin
3,Bitcoin
4,Ethereum
...,...
527,ZEPHYR
528,Gapcoin
529,Beldex
530,Horizen


In [45]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns=['CoinName','Unnamed: 0'])
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2000000000
1242,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,CryptoNight,PoW,9.802226e+08,1400222610
1246,Equihash,PoW,7.296538e+06,21000000


In [46]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1242,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1245,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1246,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Standardize the data with StandardScaler().
scaler = StandardScaler()
crypto_scaled_df = scaler.fit_transform(X)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [48]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [49]:
# Create a DataFrame with the three principal components.
crypto_pca = pca.fit_transform(crypto_scaled_df)
pcs_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"])
pcs_df

,PC 1,PC 2,PC 3
0,-0.340160,1.009353,-0.564642
1,-0.323478,1.009457,-0.565056
2,2.310773,1.670443,-0.636884
3,-0.145998,-1.283214,0.192660
4,-0.154052,-2.017638,0.376377
...,...,...,...
527,2.458660,0.874525,0.033171
528,-0.338202,1.009229,-0.564671
529,0.323647,-2.297084,0.425038
530,-0.146656,-2.085408,0.462847


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [50]:
# Create an elbow curve to find the best value for K.
# Initialize the K-Means model.
inertia = []
k = list(range(1,11))

# Fit the model
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(pcs_df)
    inertia.append(model.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

c:\Users\TBurnham-Laptop\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [66]:
# Predict clusters
predictions = model.predict(pcs_df)
predictions_df= pd.DataFrame(data=predictions, columns=['Predictions'])
predictions_df

,Predictions
0,9
1,9
2,6
3,8
4,1
...,...
527,6
528,9
529,1
530,1


In [75]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1, join='inner')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = CoinName_df.CoinName

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = predictions_df

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(205, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.340160,1.009353,-0.564642,42 Coin,9
2,Scrypt,PoW/PoS,1.055185e+09,532000000,2.310773,1.670443,-0.636884,EliteCoin,6
5,X13,PoW/PoS,2.927942e+10,314159265359,-0.170756,-1.150382,-0.004224,Litecoin,8
7,SHA-256,PoW,1.792718e+07,21000000,-0.147040,-2.188571,0.394534,Monero,1
8,Ethash,PoW,1.076842e+08,0,-0.152492,-2.017745,0.376355,Ethereum Classic,1
9,Scrypt,PoW,6.303924e+07,84000000,-0.146655,-2.085408,0.462847,ZCash,1
10,X11,PoW/PoS,9.031294e+06,22000000,-0.287001,1.633827,-0.024259,Bitshares,0
11,CryptoNight-V7,PoW,1.720114e+07,0,0.161310,-1.820476,0.375453,DigiByte,1
12,Ethash,PoW,1.133597e+08,210000000,-0.314230,0.876495,-0.367783,BitcoinDark,9
13,Equihash,PoW,7.383056e+06,21000000,-0.267149,0.655001,-0.067760,PayCoin,9


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [70]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", color="Class", symbol="Class", width=800, 
                    hover_name="CoinName", hover_data=["Algorithm"])
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [71]:
# Create a table with tradable cryptocurrencies.
tradable_crypto = clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 
                                                  'TotalCoinsMined', 'Class'], sortable=True, selectable=True)
tradable_crypto

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [72]:
# Print the total number of tradable cryptocurrencies.
print(len(clustered_df))

205


In [73]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_cluster = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']].copy()
X_cluster_scaled = MinMaxScaler().fit_transform(X_cluster)
X_cluster_scaled

array([[1.09375000e-10, 0.00000000e+00],
       [1.38541667e-03, 5.73261824e-03],
       [8.18123087e-01, 1.59069533e-01],
       [5.46875000e-05, 9.73946969e-05],
       [0.00000000e+00, 5.85027630e-04],
       [2.18750000e-04, 3.42479966e-04],
       [5.72916667e-05, 4.90650729e-05],
       [0.00000000e+00, 9.34503042e-05],
       [5.46875000e-04, 6.15861433e-04],
       [5.46875000e-05, 4.01105093e-05],
       [9.37648568e-03, 1.48944271e-02],
       [5.46875000e-02, 6.19678145e-02],
       [5.72916667e-05, 7.00191343e-06],
       [3.25520833e-05, 6.51681401e-05],
       [5.46875000e-05, 3.13379007e-05],
       [9.11458333e-04, 1.38767500e-04],
       [5.20833333e-05, 6.05741516e-05],
       [1.66666667e-04, 6.63115322e-05],
       [4.36681875e-05, 9.85243138e-05],
       [0.00000000e+00, 3.45213672e-03],
       [5.20833333e-03, 9.17196335e-03],
       [1.00000000e+00, 2.49909484e-10],
       [4.42176010e-04, 9.24687783e-04],
       [6.51041667e-04, 7.00511385e-04],
       [2.604166

In [76]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
sc_df = pd.DataFrame(X_cluster_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
sc_df['CoinName'] = clustered_df.CoinName

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
sc_df['Class'] = clustered_df.Class

sc_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,1.093750e-10,0.000000,42 Coin,9
2,1.385417e-03,0.005733,EliteCoin,6
5,8.181231e-01,0.159070,Litecoin,8
7,5.468750e-05,0.000097,Monero,1
8,0.000000e+00,0.000585,Ethereum Classic,1
9,2.187500e-04,0.000342,ZCash,1
10,5.729167e-05,0.000049,Bitshares,0
11,0.000000e+00,0.000093,DigiByte,1
12,5.468750e-04,0.000616,BitcoinDark,9
13,5.468750e-05,0.000040,PayCoin,9


In [77]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
sc_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', hover_cols='CoinName', by='Class')

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)